In [1]:
import cv2
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential

from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from tensorflow.keras.optimizers import SGD, Adam




In [2]:
train_data_gen=ImageDataGenerator(rescale=1./255)
validation_data_gen=ImageDataGenerator(rescale=1./255)

In [3]:
train_generator=train_data_gen.flow_from_directory("C:/Users/HP/Desktop/AI/images/train/",
target_size=(48,48),batch_size=64,color_mode="grayscale",class_mode="categorical")

validation_generator=train_data_gen.flow_from_directory("C:/Users/HP/Desktop/AI/images/validation/",
target_size=(48,48),batch_size=64,color_mode="grayscale",class_mode="categorical")



Found 28821 images belonging to 7 classes.
Found 7066 images belonging to 7 classes.


In [4]:
emotion_model=Sequential()

In [5]:
emotion_model.add(Conv2D(32,kernel_size=(3,3),activation="relu",input_shape=(48,48,1)))
emotion_model.add(Conv2D(64,kernel_size=(3,3),activation="relu"))
emotion_model.add(MaxPooling2D(pool_size=(2,2)))
emotion_model.add(Dropout(0.25))

emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))

emotion_model.add(Flatten())
emotion_model.add(Dense(1024, activation='relu'))
emotion_model.add(Dropout(0.5))
emotion_model.add(Dense(7, activation='softmax'))



In [6]:
cv2.ocl.setUseOpenCL(False)


In [7]:
emotion_model.compile(loss="categorical_crossentropy",optimizer=Adam(learning_rate=0.0001, decay=1e-6), metrics=["accuracy"])

In [8]:
emotion_model_info=emotion_model.fit_generator(
train_generator,steps_per_epoch=10,epochs=30,validation_data=validation_generator,
    validation_steps=10)

model_json=emotion_model.to_json()

with open("emotion_model.json","w") as json_file:
    json_file.write(model_json)
    
emotion_model.save_weights("emotion_model.h5")

D:\anaconda\lib\site-packages\tensorflow\python\keras\engine\training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/30
10/10 [==============================] - 52s 5s/step - loss: 1.8900 - accuracy: 0.2219 - val_loss: 1.8642 - val_accuracy: 0.2656
Epoch 2/30
10/10 [==============================] - 39s 4s/step - loss: 1.8341 - accuracy: 0.2656 - val_loss: 1.8252 - val_accuracy: 0.2859
Epoch 3/30
10/10 [==============================] - 35s 4s/step - loss: 1.8559 - accuracy: 0.2188 - val_loss: 1.8374 - val_accuracy: 0.2531
Epoch 4/30
10/10 [==============================] - 32s 3s/step - loss: 1.8264 - accuracy: 0.2250 - val_loss: 1.8355 - val_accuracy: 0.2406
Epoch 5/30
10/10 [==============================] - 30s 3s/step - loss: 1.8219 - accuracy: 0.2516 - val_loss: 1.8342 - val_accuracy: 0.2484
Epoch 6/30
10/10 [==============================] - 28s 3s/step - loss: 1.8105 - accuracy: 0.2734 - val_loss: 1.8227 - val_accuracy: 0.2656
Epoch 7/30
10/10 [==============================] - 26s 3s/step - loss: 1.8340 - accuracy: 0.2219 - val_loss: 1.8335 - val_accuracy: 0.2484
Epoch 8/30
10/10 [==

In [12]:
import cv2
import numpy as np
import tensorflow
from tensorflow.keras.models import model_from_json

emotion_dict = {0: "Angry", 1: "Disgusted", 2: "Fearful", 3: "Happy", 4: "Neutral", 5: "Sad", 6: "Surprised"}

# load json and create model
json_file=open("emotion_model.json","r")
loaded_model_json=json_file.read()
json_file.close()
emotion_model=model_from_json(loaded_model_json)


# load weights into new model
emotion_model.load_weights("emotion_model.h5")
print("loaded model disk")

# start the webcam feed
cap = cv2.VideoCapture(0)

# cap = cv2.VideoCapture("C:\\Users\\HP\\Desktop\\AI\\CNN\\emotion_detection\\1987949503.mp4")

while True:
    # find haar cscade to draw bounding box around face
    ret,frame=cap.read()
    frame=cv2.resize(frame,(1280,720))

    if not ret:
        break
    face_detector=cv2.CascadeClassifier("haarcascades/haarcascade_frontalface_default.xml")
    gray_frame=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)


    # detect faces available on camera
    num_faces=face_detector.detectMultiScale(gray_frame,scaleFactor=1.3,minNeighbors=5)

    # take each face available on the camera and preprocess it
    for (x,y,w,h) in num_faces:
        cv2.rectangle(frame, (x, y - 50), (x + w, y + h + 10), (0, 255, 0), 4)
        roi_gray_frame = gray_frame[y:y + h, x:x + w]
        cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray_frame, (48, 48)), -1), 0)

        # predict the emotions
        emotion_prediction = emotion_model.predict(cropped_img)
        maxindex = int(np.argmax(emotion_prediction))
        cv2.putText(frame, emotion_dict[maxindex], (x + 5, y - 20), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2,
                    cv2.LINE_AA)

    cv2.imshow('Emotion Detection', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()



loaded model disk
